
# 回帰分析

## ライブラリなど読み込み

In [ ]:
import numpy as np               
import matplotlib.pyplot as plt  
import pandas as pd

## Data: Income vs Life.Expectancy (2018)

In [ ]:
# データ読み込み
dat = np.array(pd.read_csv('data/factfulness.csv').values[:,1:3]).astype('float64')
X = dat[:,0].reshape(-1,1) # データ行列
y = dat[:,1]                          # 観測ベクトル

In [ ]:
X

## データプロット

In [ ]:
plt.xlabel('Income'); plt.ylabel("Life Expectancy")
plt.xlim(np.min(X), np.max(X))
plt.xscale("log")      # x軸をlog プロット
plt.scatter(X, y, s=10, c='blue')       # データ点のプロット
plt.show()

## 最小2乗法による推定：
sklearn.linear_model.LinearRegression

In [ ]:
# モジュール読込み
from sklearn import linear_model

lf = linear_model.LinearRegression()     # 線形モデルの設定
lf.fit(np.log(X), y)                                     # 最小２乗法： 対数をとったデータに適用
np.array([lf.intercept_,  lf.coef_])         # 切片， 傾き

## 予測値の計算

In [ ]:
Xt = np.linspace(np.min(X),np.max(X),100).reshape(-1,1) # テスト点の生成
py = lf.predict(np.log(Xt))   # 予測値

## プロット：推定結果

In [ ]:
plt.xlabel('Income'); plt.ylabel("Life Expectancy")
plt.xlim(np.min(X), np.max(X))
plt.xscale("log")                                # log プロット
plt.scatter(X, y, s=10, c='blue')      # データ点のプロット
plt.plot(Xt, py,'r-',lw=3)                   # 予測値のプロット
plt.show()

## リッジ回帰

## データ

In [ ]:
n = 100            # データ数
x = np.random.uniform(-2,2,n); ord = np.argsort(x)
y = np.sin(2*np.pi*x)/x + np.random.normal(scale=0.5,size=n)

## プロット

In [ ]:
plt.scatter(x, y,c='blue',s=10)            # データ点プロット
plt.show()

## 線形回帰モデル：多項式回帰

In [ ]:
# データ行列
degree = 10
X = np.power.outer(x,np.arange(1,degree+1))  #  基底関数：多項式
X.shape

## リッジ回帰：
sklearn.linear_model.Ridge

In [ ]:
# モジュール読込み
from sklearn import linear_model
rfit = linear_model.Ridge(alpha=0.1).fit(X,y)       # リッジ回帰(alpha = 0.1)

In [ ]:
rfit.intercept_   # 切片 w0 

In [ ]:
rfit.coef_           # 係数 w1

## 予測

In [ ]:
tx = np.linspace(-1.8, 1.8, 100)                               # テスト点
tX = np.power.outer(tx,np.arange(1,degree+1))   # テスト点に対応するデータ行列

In [ ]:
# 予測
py = rfit.predict(tX)

## プロット

In [ ]:
plt.scatter(x,y,c='blue',s=10)          # データ点のプロット
plt.plot(tx,py)
plt.show()

## プロット：正則化パラメータ alpha ごとの結果

In [ ]:
a  = np.array([0, 2**(-6), 1, 2**6])      # 正則化パラメータの候補
ls = ['-', '--', '-.', ':']                                  # プロットのラインスタイル
for i in np.arange(a.size):
    rfit = linear_model.Ridge(alpha=a[i]).fit(X,y)       # リッジ回帰
    py = rfit.predict(tX)
    # alpha 毎に，リッジ回帰の結果をプロット  
    plt.plot(tx,py,label="alpha: "+str(round(a[i], 3)), ls=ls[i], lw=2)
plt.scatter(x,y,c='black',s=10)              # データ点プロット
plt.legend();plt.show()

## 交差検証法

## データ

In [ ]:
n = 100            # データ数
x = np.random.uniform(-2,2,n); ord = np.argsort(x)
y = np.sin(2*np.pi*x)/x + np.random.normal(scale=0.5,size=n)

In [ ]:
plt.scatter(x,y); plt.show()

## いろいろな正則化パラメータでリッジ回帰

In [ ]:
degree = 15     # 15次多項式モデルを使う
X = np.power.outer(x,np.arange(1,degree+1))  #  基底関数：多項式

alphas  = 10.**np.arange(-13,5)        # 正則化パラメータの候補
alphas

In [ ]:
for a in alphas:
    rd = linear_model.Ridge(alpha=a).fit(X,y)       # リッジ回帰
    py = rd.predict(X)
    # alpha 毎に，リッジ回帰の結果をプロット  
    plt.plot(np.sort(x), py[ord], ls='-', lw=2)
plt.scatter(x, y, c='black', s=10)
plt.show()

## 交差検証法：
sklearn.linear_model.RidgeCV

In [ ]:
print('alphas: ',alphas)

In [ ]:
cvf = linear_model.RidgeCV(cv=10, alphas=alphas)  # モデル設定：リッジ回帰の交差検証法
cvf.fit(X,y)       # 学習

In [ ]:
# 最適な alpha (候補：10^(-13),...,10^4)
cvf.alpha_

## 最適な alpha によるリッジ回帰

In [ ]:
tx = np.linspace(-2,2, 100)                                       # テスト点
tX = np.power.outer(tx,np.arange(1,degree+1))   # テスト点に対応するデータ行列
py = cvf.predict(tX)          # 予測
plt.plot(tx,py,label="optimal alpha", ls='-', lw=2)
plt.scatter(x,y,c='black',s=10)              # データ点プロット
plt.legend(); plt.show()